<a href="https://colab.research.google.com/github/felanbi/Projet-OC-Categoriser-automatiquement-des-questions/blob/main/P5_02_notebooktest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import nltk
import pickle
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
data = pd.read_pickle("drive/MyDrive/data/data_tokenized_sw_free_v2.pkl")
data.head()

,post,tags
Id,,
65526399,node taking hour employee recordsgroovy xmlbel...,[javascript]
65526401,pythonuser usingibased game game begin create ...,[python]
65526407,remove ver powered whmcomplete solution whmcsi...,"[php, css]"
65526419,object allocated allocate constructremoved des...,[c++]
65526447,react route change createbrowserhistory blockr...,"[javascript, reactjs]"


In [33]:
n = 5000

df_tmp = data[data.tags.map(len) > 1].copy()
df_tmp["tags_str"] = df_tmp.tags.map(lambda x: " ".join(x))
df_tmp = df_tmp[df_tmp.tags_str.str.contains("javascript")].head(n)
df_tmp.drop(columns = ["tags_str"], inplace = True)

df_sampled = pd.DataFrame()
df_sampled = pd.concat([df_sampled, df_tmp])

df_tmp = data[data.tags.map(len) > 1].copy()
df_tmp["tags_str"] = df_tmp.tags.map(lambda x: " ".join(x))
df_tmp = df_tmp[df_tmp.tags_str.str.contains("javascript") == False]
df_tmp = df_tmp[df_tmp.tags_str.str.contains("pandas")]
df_tmp = df_tmp.head(n)
df_tmp.drop(columns = ["tags_str"], inplace = True)

df_sampled = pd.concat([df_sampled, df_tmp])

df_tmp = data[data.tags.map(len) == 1].copy()

for tag in [x for x in mlb.classes_ if x not in {"javascript", "pandas", "python"}]:
  df_tmp_tmp = df_tmp.copy()
  df_tmp_tmp["tags_str"] = df_tmp_tmp.tags.map(lambda x: " ".join(x))

  if tag in {"r", "java"}:
    df_tmp_tmp = df_tmp_tmp[df_tmp_tmp.tags_str == tag]
  else : 
    df_tmp_tmp = df_tmp_tmp[df_tmp_tmp.tags_str.str.contains(tag, regex = False)]

  df_tmp_tmp = df_tmp_tmp.head(n)
  df_tmp_tmp.drop(columns = ["tags_str"], inplace = True)
  df_sampled = pd.concat([df_sampled, df_tmp_tmp])

In [36]:
from collections import Counter
l = []

for t in df_sampled.tags:
  l.extend(t)

Counter(l).most_common()

[('reactjs', 7227),
 ('node.js', 6128),
 ('angular', 5276),
 ('php', 5123),
 ('python-3.x', 5084),
 ('amazon-web-services', 5054),
 ('java', 5043),
 ('r', 5040),
 ('android', 5039),
 ('c#', 5038),
 ('flutter', 5011),
 ('c++', 5005),
 ('javascript', 5000),
 ('pandas', 5000),
 ('python', 4793),
 ('typescript', 4506),
 ('swift', 4428),
 ('ios', 2864),
 ('css', 2576),
 ('html', 2390)]

In [34]:
df_sampled.shape

(83991, 2)

In [37]:
vocabulary = pickle.load(open("drive/MyDrive/data/vocabulary.pkl", "rb"))

In [40]:
from gensim.models import TfidfModel
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import matplotlib.pyplot as plt

texts = df_sampled.post.str.split().to_list()
# id2word = corpora.Dictionary(texts)
# id2word.filter_extremes(keep_n = 10000)

id2word = corpora.Dictionary()
id2word.add_documents([[x] for x in vocabulary])

bow_corpus = [id2word.doc2bow(text) for text in texts]
tfidf = TfidfModel(bow_corpus)
tfidf_corpus = [tfidf[text] for text in bow_corpus]

model = LdaMulticore(corpus = tfidf_corpus,
                    id2word = id2word,
                    num_topics = 20, # nombre de tags retenus
                    random_state = 42,
                    passes = 10,
                    workers = 7,
                    minimum_probability = 0)

coherencemodel = CoherenceModel(model = model, 
                                texts = texts,
                                dictionary = id2word, 
                                coherence = 'c_v')

coherencemodel.get_coherence()

0.5596721807968452

In [43]:
pickle.dump(model, open("drive/MyDrive/data/lda_model_on_sample.pkl", "wb"))

In [41]:
from pprint import pprint

print('\nPerplexity: ', model.log_perplexity(tfidf_corpus))

model_topics = model.show_topics(formatted=False)
pprint(model.print_topics(num_words=30))


Perplexity:  -6.683315847366518
[(0,
  '0.060*"item" + 0.059*"table" + 0.036*"form" + 0.030*"td" + 0.026*"material" '
  '+ 0.025*"ui" + 0.019*"mat" + 0.019*"th" + 0.015*"modal" + 0.015*"div" + '
  '0.014*"field" + 0.013*"select" + 0.013*"cell" + 0.012*"component" + '
  '0.012*"writeline" + 0.012*"control" + 0.012*"option" + 0.012*"sort" + '
  '0.011*"input" + 0.011*"ng" + 0.010*"button" + 0.010*"column" + '
  '0.009*"delete" + 0.009*"row" + 0.009*"tr" + 0.008*"list" + 0.008*"label" + '
  '0.007*"array" + 0.007*"formcontrol" + 0.007*"custom"'),
 (1,
  '0.111*"aws" + 0.031*"ec" + 0.024*"access" + 0.023*"role" + 0.022*"cluster" '
  '+ 0.022*"resource" + 0.019*"bucket" + 0.016*"deploy" + 0.015*"arn" + '
  '0.014*"instance" + 0.014*"secret" + 0.013*"allow" + 0.013*"composer" + '
  '0.012*"create" + 0.011*"cache" + 0.011*"service" + 0.011*"cloudformation" + '
  '0.010*"api" + 0.010*"cdk" + 0.010*"created" + 0.009*"region" + '
  '0.009*"running" + 0.009*"east" + 0.009*"vpc" + 0.008*"key" + 0

In [44]:
def format_topics_sentences(ldamodel, corpus):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=model, corpus=tfidf_corpus)
df_topic_sents_keywords

df_topic_sents_keywords.index = df_sampled.index
df_topic_sents_keywords.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords
Id,,,
65526447,11.0,0.8317,"react, component, const, div, prop, usestate, ..."
65526570,19.0,0.4758,"div, px, font, color, style, text, span, butto..."
65526605,19.0,0.7074,"div, px, font, color, style, text, span, butto..."
65526623,11.0,0.5443,"react, component, const, div, prop, usestate, ..."
65526847,0.0,0.7433,"item, table, form, td, material, ui, mat, th, ..."


In [46]:
df_tmp = df_topic_sents_keywords.copy()
df_tmp.sort_values(by = ["Perc_Contribution"], ascending = False, inplace = True)
df_tmp.drop_duplicates(subset = ["Dominant_Topic"], inplace = True)
df_tmp = df_tmp.join(df_sampled[["tags"]])

df_tmp[["Dominant_Topic", "Topic_Keywords", "tags"]]

,Dominant_Topic,Topic_Keywords,tags
Id,,,
66005847,8.0,"child, foo, context, flutter, dart, bar, build...",[flutter]
60090428,15.0,"node, module, js, npm, modules, run, install, ...",[reactjs]
61418553,9.0,"const, log, await, err, async, res, response, ...",[node.js]
60020358,10.0,"val, fun, num, extension, level, compiler, seq...",[swift]
59662531,11.0,"react, component, const, div, prop, usestate, ...",[reactjs]
60916674,6.0,"query, password, login, php, db, auth, request...",[php]
60075022,13.0,"google, studio, activity, visual, layout, play...",[android]
59698091,18.0,"string, self, var, public, print, nil, view, f...",[swift]
63939822,14.0,"py, python, site, install, usr, package, packa...",[python-3.x]


In [ ]:
lda_model = model
def predict_unsupervised_tags(text):
    """
    Predict tags of a preprocessed text
    
    Args:
        text(list): preprocessed text
        
    Returns:
        relevant_tags(list): list of tags
    """
    
    corpus_new = id2word.doc2bow(text)
    topics = lda_model.get_document_topics(corpus_new)
    
    #find most relevant topic according to probability
    relevant_topic = topics[0][0]
    relevant_topic_prob = topics[0][1]
    
    for i in range(len(topics)):
        if topics[i][1] > relevant_topic_prob:
            relevant_topic = topics[i][0]
            relevant_topic_prob = topics[i][1]
            
    #retrieve associated to topic tags present in submited text
    potential_tags = lda_model.get_topic_terms(topicid=relevant_topic, topn=20)
    
    relevant_tags = [id2word[tag[0]] for tag in potential_tags if id2word[tag[0]] in text]
    
    return relevant_tags

for i in range(20):
    print(f"Liste de tags à prédire : {df.tags.iloc[i]}\n")
    print(f"Liste des tags prédits par le modèle non supervisé: {predict_unsupervised_tags(df.post.iloc[i])}\n\n")

# ajouter les sorties supervisées

Liste de tags à prédire : ['javascript']

Liste des tags prédits par le modèle non supervisé: ['response', 'message', 'function']


Liste de tags à prédire : ['python']

Liste des tags prédits par le modèle non supervisé: ['print', 'self', 'def', 'len', 'input']


Liste de tags à prédire : ['php', 'css']

Liste des tags prédits par le modèle non supervisé: ['found', 'version']


Liste de tags à prédire : ['c++']

Liste des tags prédits par le modèle non supervisé: ['std']


Liste de tags à prédire : ['javascript', 'reactjs']

Liste des tags prédits par le modèle non supervisé: ['react', 'const', 'prop', 'useeffect']


Liste de tags à prédire : ['java']

Liste des tags prédits par le modèle non supervisé: ['string', 'public', 'private', 'int', 'set', 'following']


Liste de tags à prédire : ['python']

Liste des tags prédits par le modèle non supervisé: ['extension']


Liste de tags à prédire : ['ios', 'flutter']

Liste des tags prédits par le modèle non supervisé: ['firebase', 'dev', '

In [ ]:
# retirer les tfidf faibles
# les afficher